# CIFAR-10 Image Separation Model
Progetto svolto da Alex Rossi 0001089916

In [3]:
# import vari
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
import numpy as np
from matplotlib import pyplot as plt

## Caricamento e Preprocessing dati
(Già forniti)

In [3]:

(cifar10_x_train, cifar10_y_train), (cifar10_x_test, cifar10_y_test) = cifar10.load_data()
assert cifar10_x_train.shape == (50000, 32, 32, 3)
assert cifar10_x_test.shape == (10000, 32, 32, 3)
assert cifar10_y_train.shape == (50000, 1)
assert cifar10_y_test.shape == (10000, 1)

classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

cifar10_x_train = (cifar10_x_train/255.).astype(np.float32)
cifar10_x_test = (cifar10_x_test/255.).astype(np.float32)

# Separazione delle immagini in due gruppi in base alla loro etichetta
cond_1 = cifar10_y_train[:,0] < 5
cifar10_x_train_1 = cifar10_x_train[cond_1]
cifar10_y_train_1 = cifar10_y_train[cond_1]

cond_2 = cifar10_y_train[:,0] >= 5
cifar10_x_train_2 = cifar10_x_train[cond_2]
cifar10_y_train_2 = cifar10_y_train[cond_2]

cond_1_test = cifar10_y_test[:,0] < 5
cifar10_x_test_1 = cifar10_x_test[cond_1_test]
cifar10_y_test_1 = cifar10_y_test[cond_1_test]

cond_2_test = cifar10_y_test[:,0] >= 5
cifar10_x_test_2 = cifar10_x_test[cond_2_test]
cifar10_y_test_2 = cifar10_y_test[cond_2_test]

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


### Definizione generatore immagini
(**Nota**: una parte del formato dei dati è stato cambiato ai fini di rendere il tutto compatibile con il modello creato, in modo tale da non restituire errori durante la fase di training)

In [10]:
def datagenerator(X1, X2, Y1, Y2, batchsize):
    size1 = X1.shape[0]
    size2 = X2.shape[0]
    Y1_cat = tf.keras.utils.to_categorical(Y1, num_classes=5)
    Y2_cat = tf.keras.utils.to_categorical(Y2-5, num_classes=5)

    while True:
        num1 = np.random.randint(0, size1, batchsize)
        num2 = np.random.randint(0, size2, batchsize)
        x_data = (X1[num1] + X2[num2]) / 2.0
        y_data = (Y1_cat[num1], Y2_cat[num2])  # Usa una tupla invece di una lista

        yield x_data, y_data

## Definizione del Modello predittorio
Per il problema proposto si è scelto l'uso di una **CNN**, (rete convoluzionale).
Si è optato per un modello CNN poichè il loro uso è tipico nei problemi di riconoscimento di immagini (Image Classification & Detection).  
Nella creazione del modello ci si è ispirati all'architettura di uno dei più famosi, _AlexNet_ (Uso di _layer convoluzionali_ seguiti da _normalizzazioni_ e _pooling_, _dropout_ dopo _layer densi_, etc...), ma facendo aggiunte come il _calcolo residuale_.

Aspetti interessanti implementati nel modello:
- Vari **blocchi convuluzioniali** con **periodiche normalizzazioni** (contrastare oerfitting);
- Uso della tecnica del **calcolo residuale** (in questa versione, rispetto ad un precedente modello implementato in cui non era presente, si ha un incremento dell'accurancy e diminuzione della loss di almeno il 10%);
- Uso di **GlobalAveragePooling()** invece di **Flatten()** per il passaggio da vettore multidimensionale a monodimensionale;
- **Layer di Dropout**: tecnica di regolarizzazione utilizzata nelle reti neurali per ridurre l'overfitting (e quindi evitare che il modello si adatti troppo ai dati di allenamento e perda la capacità di generalizzare su nuovi dati);
- Uso dell'optimizer **Adam**.

In [34]:
from tensorflow.keras import regularizers
def create_cifar10_model():
    input_layer = layers.Input(shape=(32, 32, 3))

    # Blocchi convoluzionali
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
    x = layers.BatchNormalization()(x) #batch normalization
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Residual Connection
    residual = x
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, residual])

    x = layers.GlobalAveragePooling2D()(x) # al posto di Flatten()

    # Dense layers con dropout
    dense_shared = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    dense_shared = layers.Dropout(0.5)(dense_shared)

    output_1 = layers.Dense(5, activation='softmax', name="output_1")(dense_shared)
    output_2 = layers.Dense(5, activation='softmax', name="output_2")(dense_shared)

    # Creazione del modello
    model = Model(inputs=input_layer, outputs=[output_1, output_2])

    # Compilazione con learning rate scheduling
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=['categorical_crossentropy', 'categorical_crossentropy'],
                  metrics=['accuracy', 'accuracy'])

    return model

model = create_cifar10_model()
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 32, 32, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 32, 32, 32)     │            896 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_8     │ (None, 32, 32, 32)     │            128 │ conv2d_8[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_8           │ (None, 16, 16, 32)     │              0 │ batch_normalization_8… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)         │ (None, 16, 16, 64)     │         18,496 │ max_pooling2d_8[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_9     │ (None, 16, 16, 64)     │            256 │ conv2d_9[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_9           │ (None, 8, 8, 64)       │              0 │ batch_normalization_9… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_10 (Conv2D)        │ (None, 8, 8, 128)      │         73,856 │ max_pooling2d_9[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_10    │ (None, 8, 8, 128)      │            512 │ conv2d_10[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_10          │ (None, 4, 4, 128)      │              0 │ batch_normalization_1… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_11 (Conv2D)        │ (None, 4, 4, 128)      │        147,584 │ max_pooling2d_10[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_11    │ (None, 4, 4, 128)      │            512 │ conv2d_11[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 4, 4, 128)      │              0 │ batch_normalization_1… │
│                           │                        │                │ max_pooling2d_10[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 128)            │              0 │ add[0][0]              │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)      

 Total params: 277,834 (1.06 MB)

 Trainable params: 277,130 (1.06 MB)

 Non-trainable params: 704 (2.75 KB)

## Training modello

In [35]:
# Mini test modello
datagen_train = datagenerator(cifar10_x_train_1, cifar10_x_train_2, cifar10_y_train_1, cifar10_y_train_2, batchsize=64)
x_batch, y_batch = next(datagen_train)

print("x_batch shape:", x_batch.shape)  # Dovrebbe essere (64, 32, 32, 3)
print("y_batch[0] shape:", y_batch[0].shape)  # Dovrebbe essere (64, 5)
print("y_batch[1] shape:", y_batch[1].shape)  # Dovrebbe essere (64, 5)



x_batch shape: (64, 32, 32, 3)
y_batch[0] shape: (64, 5)
y_batch[1] shape: (64, 5)


Per cercare di contrastare l'overfitting, si usufruisce di un meccanismo di **early stopping**: si tiene sotto controllo la _loss_ e se dopo _3 epoche_ consecutive non vi è miglioramento, e quindi non c'è un apprendimento da parte del modello, fermo il training stesso.

In [36]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)


Salvo i progressi del modello tramite **checkpoint** tenendo traccia della _loss_: se vi è una diminuzione rispetto all'epoca precedente, salvo i progressi. 

In [37]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(
    'model_checkpoint.keras',
    monitor='loss',
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    verbose=1
)

Primo training: inizio con una batch_size 'piccola' per poi anando ad aumentare durante le altre fasi di training del modello.

In [38]:
batch_size = 64

datagen_train = datagenerator(cifar10_x_train_1, cifar10_x_train_2, cifar10_y_train_1, cifar10_y_train_2, batchsize=batch_size)

# Training del modello
history = model.fit(
    datagen_train,
    steps_per_epoch=1000,
    epochs=100,
    callbacks=[early_stopping, checkpoint]
    )


Epoch 1/100
 998/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.5487 - output_1_accuracy: 0.4056 - output_1_loss: 1.4038 - output_2_accuracy: 0.4684 - output_2_loss: 1.3100
Epoch 1: loss improved from inf to 2.91562, saving model to model_checkpoint.keras
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 3.5468 - output_1_accuracy: 0.4058 - output_1_loss: 1.4035 - output_2_accuracy: 0.4686 - output_2_loss: 1.3096
Epoch 2/100
 995/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.2425 - output_1_accuracy: 0.5131 - output_1_loss: 1.1891 - output_2_accuracy: 0.6231 - output_2_loss: 0.9788
Epoch 2: loss improved from 2.91562 to 2.18638, saving model to model_checkpoint.keras
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 2.2422 - output_1_accuracy: 0.5132 - output_1_loss: 1.1889 - output_2_accuracy: 0.6232 - output_2_loss: 0.9787
Epoch 3/100
 995/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.0498 - output_1_accuracy: 0.5576 - output_1_loss: 1.0979 - output_2_accuracy: 0.6673 - outpu

Si può notare come viene attuata _l'Early stopping_, ma siccome _loss_ e _accuracy degli output_ non sono ancora soddisfacenti ripeto più volte il training effettuando del tuning.

In [39]:
from tensorflow.keras.models import load_model

In [43]:
model = load_model('model_checkpoint.keras')
batch_size = 256

datagen_train = datagenerator(cifar10_x_train_1, cifar10_x_train_2, cifar10_y_train_1, cifar10_y_train_2, batchsize=batch_size)

# Training del modello
history = model.fit(
    datagen_train,
    steps_per_epoch=2000,
    epochs=50,
    callbacks=[early_stopping, checkpoint]
    )

Epoch 1/50
1999/2000 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 1.1296 - output_1_accuracy: 0.7648 - output_1_loss: 0.6308 - output_2_accuracy: 0.8455 - output_2_loss: 0.4335
Epoch 1: loss improved from 1.20950 to 1.12628, saving model to model_checkpoint.keras
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 36s 15ms/step - loss: 1.1296 - output_1_accuracy: 0.7648 - output_1_loss: 0.6308 - output_2_accuracy: 0.8455 - output_2_loss: 0.4335
Epoch 2/50
1997/2000 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.1231 - output_1_accuracy: 0.7654 - output_1_loss: 0.6282 - output_2_accuracy: 0.8468 - output_2_loss: 0.4317
Epoch 2: loss improved from 1.12628 to 1.11922, saving model to model_checkpoint.keras
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 28s 14ms/step - loss: 1.1231 - output_1_accuracy: 0.7654 - output_1_loss: 0.6282 - output_2_accuracy: 0.8468 - output_2_loss: 0.4317
Epoch 3/50
1997/2000 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.1140 - output_1_accuracy: 0.7668 - output_1_loss: 0.6234 - output_2_accuracy: 0.8486 

In [45]:
model = load_model('model_checkpoint.keras')
batch_size = 512

datagen_train = datagenerator(cifar10_x_train_1, cifar10_x_train_2, cifar10_y_train_1, cifar10_y_train_2, batchsize=batch_size)

# Training del modello
history = model.fit(
    datagen_train,
    steps_per_epoch=1000,
    epochs=50,
    callbacks=[early_stopping, checkpoint]
    )

Epoch 1/50
 999/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 1.0572 - output_1_accuracy: 0.7742 - output_1_loss: 0.6037 - output_2_accuracy: 0.8569 - output_2_loss: 0.4037
Epoch 1: loss improved from 1.06794 to 1.05866, saving model to model_checkpoint.keras
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 29s 24ms/step - loss: 1.0572 - output_1_accuracy: 0.7742 - output_1_loss: 0.6037 - output_2_accuracy: 0.8569 - output_2_loss: 0.4037
Epoch 2/50
 999/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 1.0544 - output_1_accuracy: 0.7757 - output_1_loss: 0.6014 - output_2_accuracy: 0.8578 - output_2_loss: 0.4015
Epoch 2: loss improved from 1.05866 to 1.05320, saving model to model_checkpoint.keras
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 24s 24ms/step - loss: 1.0544 - output_1_accuracy: 0.7757 - output_1_loss: 0.6014 - output_2_accuracy: 0.8578 - output_2_loss: 0.4015
Epoch 3/50
 999/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 1.0509 - output_1_accuracy: 0.7775 - output_1_loss: 0.5968 - output_2_accuracy: 0.8573 

In [46]:
model = load_model('model_checkpoint.keras')
batch_size = 1024

datagen_train = datagenerator(cifar10_x_train_1, cifar10_x_train_2, cifar10_y_train_1, cifar10_y_train_2, batchsize=batch_size)

# Training del modello
history = model.fit(
    datagen_train,
    steps_per_epoch=1000,
    epochs=20,
    callbacks=[early_stopping, checkpoint]
    )

Epoch 1/20
 999/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 1.0021 - output_1_accuracy: 0.7869 - output_1_loss: 0.5743 - output_2_accuracy: 0.8642 - output_2_loss: 0.3830
Epoch 1: loss improved from 1.04805 to 0.99414, saving model to model_checkpoint.keras
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 55s 45ms/step - loss: 1.0021 - output_1_accuracy: 0.7869 - output_1_loss: 0.5743 - output_2_accuracy: 0.8642 - output_2_loss: 0.3830
Epoch 2/20
 999/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.9869 - output_1_accuracy: 0.7893 - output_1_loss: 0.5684 - output_2_accuracy: 0.8665 - output_2_loss: 0.3767
Epoch 2: loss improved from 0.99414 to 0.98502, saving model to model_checkpoint.keras
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 45s 45ms/step - loss: 0.9869 - output_1_accuracy: 0.7893 - output_1_loss: 0.5684 - output_2_accuracy: 0.8665 - output_2_loss: 0.3767
Epoch 3/20
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.9845 - output_1_accuracy: 0.7895 - output_1_loss: 0.5680 - output_2_accuracy: 0.8675 

In [48]:
model = load_model('model_checkpoint.keras')
batch_size = 2048

datagen_train = datagenerator(cifar10_x_train_1, cifar10_x_train_2, cifar10_y_train_1, cifar10_y_train_2, batchsize=batch_size)

# Training del modello
history = model.fit(
    datagen_train,
    steps_per_epoch=1000,
    epochs=10,
    callbacks=[early_stopping, checkpoint]
    )

Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.8550 - output_1_accuracy: 0.8181 - output_1_loss: 0.4949 - output_2_accuracy: 0.8869 - output_2_loss: 0.3221
Epoch 1: loss improved from 0.89182 to 0.85041, saving model to model_checkpoint.keras
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 91s 86ms/step - loss: 0.8550 - output_1_accuracy: 0.8181 - output_1_loss: 0.4949 - output_2_accuracy: 0.8869 - output_2_loss: 0.3221
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.8485 - output_1_accuracy: 0.8188 - output_1_loss: 0.4924 - output_2_accuracy: 0.8868 - output_2_loss: 0.3208
Epoch 2: loss improved from 0.85041 to 0.84836, saving model to model_checkpoint.keras
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 85s 85ms/step - loss: 0.8485 - output_1_accuracy: 0.8188 - output_1_loss: 0.4924 - output_2_accuracy: 0.8868 - output_2_loss: 0.3208
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.8479 - output_1_accuracy: 0.8192 - output_1_loss: 0.4921 - output_2_accuracy: 0.8875 

In [53]:
model = load_model('model_checkpoint.keras')
batch_size = 4096

datagen_train = datagenerator(cifar10_x_train_1, cifar10_x_train_2, cifar10_y_train_1, cifar10_y_train_2, batchsize=batch_size)

# Training del modello
history = model.fit(
    datagen_train,
    steps_per_epoch=3000,
    epochs=5,
    callbacks=[early_stopping, checkpoint]
    )

Epoch 1/5
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - loss: 0.7911 - output_1_accuracy: 0.8306 - output_1_loss: 0.4624 - output_2_accuracy: 0.8949 - output_2_loss: 0.2989
Epoch 1: loss improved from 0.82635 to 0.78912, saving model to model_checkpoint.keras
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 520s 172ms/step - loss: 0.7911 - output_1_accuracy: 0.8306 - output_1_loss: 0.4624 - output_2_accuracy: 0.8949 - output_2_loss: 0.2989
Epoch 2/5
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - loss: 0.7847 - output_1_accuracy: 0.8323 - output_1_loss: 0.4582 - output_2_accuracy: 0.8957 - output_2_loss: 0.2967
Epoch 2: loss improved from 0.78912 to 0.78206, saving model to model_checkpoint.keras
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 519s 173ms/step - loss: 0.7847 - output_1_accuracy: 0.8323 - output_1_loss: 0.4582 - output_2_accuracy: 0.8957 - output_2_loss: 0.2967
Epoch 3/5
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - loss: 0.7753 - output_1_accuracy: 0.8343 - output_1_loss: 0.4528 - output_2_accuracy: 0.8

In [ ]:
model = load_model('model_checkpoint.keras')
batch_size = 8192

datagen_train = datagenerator(cifar10_x_train_1, cifar10_x_train_2, cifar10_y_train_1, cifar10_y_train_2, batchsize=batch_size)

# Training del modello
history = model.fit(
    datagen_train,
    steps_per_epoch=500,
    epochs=8,
    callbacks=[early_stopping, checkpoint]
    )

Epoch 1/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step - loss: 0.7370 - output_1_accuracy: 0.8424 - output_1_loss: 0.4325 - output_2_accuracy: 0.9024 - output_2_loss: 0.2782
Epoch 1: loss improved from 0.75919 to 0.73213, saving model to model_checkpoint.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 199s 344ms/step - loss: 0.7370 - output_1_accuracy: 0.8425 - output_1_loss: 0.4325 - output_2_accuracy: 0.9024 - output_2_loss: 0.2782
Epoch 2/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step - loss: 0.7296 - output_1_accuracy: 0.8430 - output_1_loss: 0.4301 - output_2_accuracy: 0.9033 - output_2_loss: 0.2759
Epoch 2: loss improved from 0.73213 to 0.72937, saving model to model_checkpoint.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 171s 342ms/step - loss: 0.7296 - output_1_accuracy: 0.8430 - output_1_loss: 0.4301 - output_2_accuracy: 0.9033 - output_2_loss: 0.2759
Epoch 3/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step - loss: 0.7305 - output_1_accuracy: 0.8430 - output_1_loss: 0.4304 - output_2_accuracy: 0.9030 - outp

Poichè i risultati di training sembrano abbastanza soddisfacenti, ma sicuramente migliorabili, si passa alla parte di testing e valutazione modello.

## Valutazione modello
Valutazione del modello svolta secondo la consegna (testing su 10000 campioni ed _evaluation_ svolta per 10 volte consecutive per calcolare la _accuracy media_ e la _standard devition_), ispirandosi al file fornito.

In [14]:
# Generatore di testing
testgen = datagenerator(cifar10_x_test_1, cifar10_x_test_2, cifar10_y_test_1, cifar10_y_test_2, batchsize=10000)
# Ricavo modello salvato
model = load_model('model_checkpoint.keras')

# Funzione per valutare il modello
def eval_model(model, testgen, repeat_eval=10):
    """
    Valuta il modello calcolando l'accuratezza media e la deviazione standard.
    """
    eval_results = []
    for _ in range(repeat_eval):
        eval_samples_x, eval_samples_y = next(testgen)
        predictions = model.predict(eval_samples_x)

        correct_1 = np.argmax(predictions[0], axis=1) == np.argmax(eval_samples_y[0], axis=1)
        correct_2 = np.argmax(predictions[1], axis=1) == np.argmax(eval_samples_y[1], axis=1)

        accuracy = (np.mean(correct_1) + np.mean(correct_2)) / 2
        eval_results.append(accuracy)

    mean_accuracy = np.mean(eval_results)
    std_deviation = np.std(eval_results)
    return mean_accuracy, std_deviation

# Valutazione del modello
mean_acc, std_dev = eval_model(model, testgen)
print(f"Mean accuracy: {mean_acc:.4f}, Standard deviation: {std_dev:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 91s 290ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Mean accuracy: 0.7689, Standard deviation: 0.0015


## Considerazioni finali e Possibili Improvement
Come si può notare dai risultati del training effettuato, il modello tende a raggiungere un'_accuracy_ sul secondo output leggermente maggiore rispetto al primo (*output_1_accuracy*: 0.8439, *output_2_accuracy*: 0.9033) con una _loss_ generale di circa 0.72. Sicuramente con ulteriori fasi di training si potrebbe ottenere un **leggero miglioramento** di qualche punto perentuale.

Come si può notare nelle ultime fasi di training, l'improvment della loss diventa sempre più bassa.
Di conseguenza si evince che un'**ulteriore aumento rilevante delle prestazioni**, rispetto alla fase di testing (es: del 10%, e quindi passare da circa 77% a circa il 90% di _mean accuracy_ come i migliori modelli per il problema proposto), può esser dato soltanto andando a **modificare il modello stesso** ed attuando alcune accortezze sui dati generati:
- **Data Augmentation**: aumentare la varietà del dataset introducendo trasformazioni come rotazioni, flip, scaling;
- **Aggiunta di ulteriori layer**: avere un'architettura più profonda;
- **Tuning del modello**: sperimentare con anche altri optimizer, ottimizzare la dimensione del batch o il tasso di dropout;
- **Regularizzazione migliore**: applicare altre tecniche di regolarizzazione come L1/L2 o aggiungere più Dropout;
- **Validation set**: utilizzare un set di validazione per monitorare l'overfitting durante il training (ad esempio il 20% dei dati ti training).

Per quanto ne concerne la _Standard deviation_ essa è abbastanza bassa, quindi si può dire che il modello è **stabile** e **coerente**.
